## 0.0 IMPORTS

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats import kurtosis, skew, norm
import math
import statistics
from scipy.stats import chi2

pd.set_option('float_format', '{:.3f}'.format)

## 0.1 - HELPER FUNCTION (FUNÇÕES AUXILIARES)

In [2]:
def show_descriptive_statistical(df):
    # Central Tendency - mean, median
    ct1 = pd.DataFrame(df.apply(np.mean)).T
    ct2 = pd.DataFrame(df.apply(np.median)).T

    # Dispersion - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(df.apply(np.std)).T
    d2 = pd.DataFrame(df.apply(min)).T
    d3 = pd.DataFrame(df.apply(max)).T
    d4 = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    return m

## 0.2 - PATH DEFINITION

In [3]:
HOME_PATH = '/home/valmir/Documentos/deteccao_fraude/'
RAW_DATA_PATH = 'data/raw/'
INTERIM_DATA_PATH = 'data/intermediario/'
FIGURE_PATH = 'reports/figures/'

## 0.3 - LOAD DATA

In [4]:
base_fraude_raw = pd.read_csv(HOME_PATH+RAW_DATA_PATH+'fraud_detection_dataset.csv')
base_fraude_raw.tail()


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362615,743,CASH_OUT,339682.130,C786484425,339682.130,0.000,C776919290,0.000,339682.130,1,0
6362616,743,TRANSFER,6311409.280,C1529008245,6311409.280,0.000,C1881841831,0.000,0.000,1,0
6362617,743,CASH_OUT,6311409.280,C1162922333,6311409.280,0.000,C1365125890,68488.840,6379898.110,1,0
6362618,743,TRANSFER,850002.520,C1685995037,850002.520,0.000,C2080388513,0.000,0.000,1,0
6362619,743,CASH_OUT,850002.520,C1280323807,850002.520,0.000,C873221189,6510099.110,7360101.630,1,0


## 1.0 - DATA DESCRIPITION

In [5]:
#base_fraude_raw1 = base_fraude_raw.copy()
#base_fraude_raw1.to_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw1.csv')
base_fraude_raw1 = pd.read_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw1.csv')


## 1.1 - DATA FIELD'S

Os dados contidos nesta base de dados são simulados e compreendem 30 dias. Estas transações podem ser de diferentes tipo e foram executadas via mobile devices. As features contidas nesta base de dados são:

 * `step` - Passo: representa o total de horas transcorrido desde o início da simulação. Esta feature vai variar entre 1 e 95(4 dias);

 * `type`: Tipo: tipo de transação(depósito, saque, débito, pagamento e transferencia);

 * `amount`: Quantia: total que foi transacionado;

 * `nameOring`: ClienteOrigem: cliente que iniciou atransação;

 * `oldbalanceOrg`: SaldoInicialOrigem: saldo da conta de origem antes da transação;

 * `newbalanceOrig`: SaldoFinalOrigem: saldo da conta de origem depois da transação;

 * `nameDest`: ClienteDestino: cliente de destino da transação;

 * `oldbalandeDestino`: SaldoInicialDestino: saldo da conta destino antes da transação;

 * _`newbalanceDest`_: SaldoFinalDestino: saldo da conta destino depois da transação;

 * `isFraud`: Fraude: flag que define se a transação é fraudolenta ou não. Nesta simulação de fraude é a assumir se 'a conta do usuario foi   esvaziada e transferindo o dinherio para outra conta e depois sacado
 
 * `IssFlaggedFraud`: SinalizadaFraude: automaticamente categorizada como fraude pelo banco por fraude por tentar trasnferir mais de 200 mil em uma única conta


## 1.2 - RENAME COLUMNS

In [6]:
base_fraude_raw1.rename(columns={'step':'Passos', 'type':'Tipo', 'amount': 'Quantia',
                                'nameOrig':'ClienteOrigem', 'oldbalanceOrg': 'SaldoInicalOrig',
                                'newbalanceOrig':'SaldoFinalOrig', 'nameDest':'ClienteDest',
                                'oldbalanceDest':'SaldoInicialDest', 'newbalanceDest':'SaldoFinalDest',
                                'isFraud':'Fraude', 'isFlaggedFraud':'SinalizadaFraude'}, inplace=True)


## 1.3 - DATA DIMENSIONS

In [7]:
num_linhas, num_colunas = base_fraude_raw1.shape

print('Numero de linhas', num_linhas)
print('Numero de colunas', num_colunas)

Numero de linhas 6362620
Numero de colunas 12


## 1.4 - DATA TYPES

In [8]:
#base_fraude_categoria = base_fraude_raw1[['Tipo','ClienteDest', 'ClienteOrigem']].astype('category')
base_fraude_raw1.dtypes

Unnamed: 0            int64
Passos                int64
Tipo                 object
Quantia             float64
ClienteOrigem        object
SaldoInicalOrig     float64
SaldoFinalOrig      float64
ClienteDest          object
SaldoInicialDest    float64
SaldoFinalDest      float64
Fraude                int64
SinalizadaFraude      int64
dtype: object

## 1.5 CHANGE TYPE

#### Não houve nenhuma mudança nos tipos de variaveis nessa etapa, pois todas as features se encontravam com seus tipos bem definidos

## 1.6 - CHECK N.A.

### Não foi encontrado nenhum valor nulo nesse Dataset

In [9]:
base_fraude_raw1.isnull().sum()

Unnamed: 0          0
Passos              0
Tipo                0
Quantia             0
ClienteOrigem       0
SaldoInicalOrig     0
SaldoFinalOrig      0
ClienteDest         0
SaldoInicialDest    0
SaldoFinalDest      0
Fraude              0
SinalizadaFraude    0
dtype: int64

In [10]:
base_fraude_raw1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Unnamed: 0        int64  
 1   Passos            int64  
 2   Tipo              object 
 3   Quantia           float64
 4   ClienteOrigem     object 
 5   SaldoInicalOrig   float64
 6   SaldoFinalOrig    float64
 7   ClienteDest       object 
 8   SaldoInicialDest  float64
 9   SaldoFinalDest    float64
 10  Fraude            int64  
 11  SinalizadaFraude  int64  
dtypes: float64(5), int64(4), object(3)
memory usage: 582.5+ MB


## 1.7 - FILL OUT N.A.

### Não havendo nenhum valor faltante nesse Dataset, não foi nescessario nenhum metodo de preenchimento ou exclusão dos mesmos

## 1.8 DISCRIPTIVE STATISTICAL

In [11]:
num_attributes = base_fraude_raw1.select_dtypes(include=['int64', 'float64'])
cat_attributes = base_fraude_raw1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])



Medidas de tendencia central - 
 - Media
 - Mediana
 

Medidas de Dispersão

- Skewness
- Kurtosis
- Desvio Padrão
- Range (alcance)
- Min
- Max

### 1.8.1 - NUMERIC ESTATISTIC

In [12]:
m = show_descriptive_statistical(num_attributes)
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,Unnamed: 0,0.000,6362619.000,6362619.000,3181309.500,3181309.500,1836730.185,0.000,-1.200
1,Passos,1.000,743.000,742.000,243.397,239.000,142.332,0.375,0.329
2,Quantia,0.000,92445516.640,92445516.640,179861.904,74871.940,603858.184,30.994,1797.957
3,SaldoInicalOrig,0.000,59585040.370,59585040.370,833883.104,14208.000,2888242.446,5.249,32.965
4,SaldoFinalOrig,0.000,49585040.370,49585040.370,855113.669,0.000,2924048.273,5.177,32.067
5,SaldoInicialDest,0.000,356015889.350,356015889.350,1100701.667,132705.665,3399179.846,19.922,948.674
6,SaldoFinalDest,0.000,356179278.920,356179278.920,1224996.398,214661.440,3674128.653,19.352,862.157
7,Fraude,0.000,1.000,1.000,0.001,0.000,0.036,27.780,769.703
8,SinalizadaFraude,0.000,1.000,1.000,0.000,0.000,0.002,630.604,397659.062


In [13]:
base_fraude_raw1.tail()

,Unnamed: 0,Passos,Tipo,Quantia,ClienteOrigem,SaldoInicalOrig,SaldoFinalOrig,ClienteDest,SaldoInicialDest,SaldoFinalDest,Fraude,SinalizadaFraude
6362615,6362615,743,CASH_OUT,339682.130,C786484425,339682.130,0.000,C776919290,0.000,339682.130,1,0
6362616,6362616,743,TRANSFER,6311409.280,C1529008245,6311409.280,0.000,C1881841831,0.000,0.000,1,0
6362617,6362617,743,CASH_OUT,6311409.280,C1162922333,6311409.280,0.000,C1365125890,68488.840,6379898.110,1,0
6362618,6362618,743,TRANSFER,850002.520,C1685995037,850002.520,0.000,C2080388513,0.000,0.000,1,0
6362619,6362619,743,CASH_OUT,850002.520,C1280323807,850002.520,0.000,C873221189,6510099.110,7360101.630,1,0


### 1.8.2 - CATEGORICAL ATRIBUTES



In [14]:
cat_attributes.apply(lambda x: x.unique().shape[0])

Tipo                   5
ClienteOrigem    6353307
ClienteDest      2722362
dtype: int64

## 2.0 - FEATURE ENGINEERING- PARTE I

In [15]:
#base_fraude_raw2 = base_fraude_raw1.copy()
#base_fraude_raw2.to_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw2.csv')
base_fraude_raw2 = pd.read_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw2.csv')

## 2.1 - MAPA MENTAL DE HIPOTESE

<img src= "/home/valmir/Documentos/deteccao_fraude/reports/figures/FRAUDE.jpg" width="1200" height= "350">


## 2.2 - CRIAÇÃO DAS HIPOTESE

### TRANSAÇÕES 
 * QUANDO OS VALORES DAS TRANSAÇÕES FOREM MUITO ALTOS OU INCOMUNS EM RELAÇÃO AOS PADRÕES COMUNS DO CLIENTE, SERAM CLASSIFICADOS COMO FRAUDES;

 * QUANDO AS TRANSAÇÕES FOREM REALIZADAS EM HORÁRIOS ATÍPICOS AOS PADRÕES COMUNS REALIZADAS PELO CLIENTE, SERAM CLASSIFICADOS COMO FRAUDES;

 * QUANDO AS TRANSAÇÕES FOREM REALIZADAS EM MOMENTOS DE TEMPO MUITO CURTOS, SERAM CLASSIFICADOS COMO FRAUDES; 

 ### COMPORTAMENTO DO CLIENTE
 * QUANDO HOUVER MUDANÇAS REPENTINAS DOS VALORES DOS GASTOS E/OU DA FORMA DE TRANSAÇÃO TÍPICA DO CLIENTE, SERAM CLASSIFICADOS COMO FRAUDES;

 * QUANDO AS TRANSAÇÕES MUDAREM REPENTINAMENTE DO TIPO DA CATEGORIA QUE O CLIENTE EVENTUALMENTE REALIZA, SERAM CLASSIFICADOS COMO FRAUDES;

 * QUANDO HOUVER A TROCA DESORDENADA DO CARTÃO QUE O CLIENTE UTILIZA EVENTUALMENTE EM SUAS TRANSAÇÕES, SERAM CLASSIFICADOS COMO FRAUDES;

 ### DADOS DE AUTENTICAÇÃO
 * QUANDO HOUVER MULTIPLAS TENTATIVAS ERRADAS, SOLICITAÇÃO DE NOVA SENHA ANTES DO CLIENTE CONSEGUIR LOGAR, SERAM CLASSIFICADOS COMO FRAUDES;
 
 ### TRANSAÇÕES SUSPEITAS
 * QUANDO HOUVER TRANSFERENCIAS PARA CONTAS DIVERSAS DAS HABITUAIS REALIZADAS PELO CLIENTE, SERAM CLASSIFICADOS COMO FRAUDES;
 
 * QUANDO HOUVER TRANSFERENCIAS PARA CONTAS INTERNACIONAIS COM FREQUENCIAS DE CURTO ESPAÇO DE TEMPO, SERAM CLASSIFICADOS COMO FRAUDES;


## 2.3 - LISTA FINAL DE HIPOTESE

### TRANSAÇÕES 
 * QUANDO OS VALORES DAS TRANSAÇÕES FOREM MUITO ALTOS OU INCOMUNS EM RELAÇÃO AOS PADRÕES COMUNS DO CLIENTE, SERAM CLASSIFICADOS COMO FRAUDES;

 * QUANDO AS TRANSAÇÕES FOREM REALIZADAS EM MOMENTOS DE TEMPO MUITO CURTOS, SERAM CLASSIFICADOS COMO FRAUDES; 

 ### COMPORTAMENTO DO CLIENTE
 * QUANDO HOUVER MUDANÇAS REPENTINAS DOS VALORES DOS GASTOS E/OU DA FORMA DE TRANSAÇÃO TÍPICA DO CLIENTE, SERAM CLASSIFICADOS COMO FRAUDES;

 ### TRANSAÇÕES SUSPEITAS
 * QUANDO HOUVER TRANSFERENCIAS PARA CONTAS DIVERSAS DAS HABITUAIS REALIZADAS PELO CLIENTE, SERAM CLASSIFICADOS COMO FRAUDES;
 
 

## 2.4 - FEATURE ENGINEERING - PARTE 1

`QUANTAS TRANSAÇÕES REALIZOU CADA CLIENTE? ` 
Criei a coluna chamada FreqCliente que nada mais é que a frequencia de transações por cliente

In [20]:
base_fraude_raw2['FreqCliente'] = base_fraude_raw2.groupby('ClienteOrigem')['ClienteOrigem'].transform('count')
base_fraude_raw2

,Unnamed: 0.1,Unnamed: 0,Passos,Tipo,Quantia,ClienteOrigem,SaldoInicalOrig,SaldoFinalOrig,ClienteDest,SaldoInicialDest,SaldoFinalDest,Fraude,SinalizadaFraude,FreqCliente
0,0,0,1,PAYMENT,9839.640,C1231006815,170136.000,160296.360,M1979787155,0.000,0.000,0,0,1
1,1,1,1,PAYMENT,1864.280,C1666544295,21249.000,19384.720,M2044282225,0.000,0.000,0,0,1
2,2,2,1,TRANSFER,181.000,C1305486145,181.000,0.000,C553264065,0.000,0.000,1,0,1
3,3,3,1,CASH_OUT,181.000,C840083671,181.000,0.000,C38997010,21182.000,0.000,1,0,1
4,4,4,1,PAYMENT,11668.140,C2048537720,41554.000,29885.860,M1230701703,0.000,0.000,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,6362615,6362615,743,CASH_OUT,339682.130,C786484425,339682.130,0.000,C776919290,0.000,339682.130,1,0,1
6362616,6362616,6362616,743,TRANSFER,6311409.280,C1529008245,6311409.280,0.000,C1881841831,0.000,0.000,1,0,1
6362617,6362617,6362617,743,CASH_OUT,6311409.280,C1162922333,6311409.280,0.000,C1365125890,68488.840,6379898.110,1,0,1
6362618,6362618,6362618,743,TRANSFER,850002.520,C1685995037,850002.520,0.000,C2080388513,0.000,0.000,1,0,1


## 3.0 - FILTRAGEM DE VARIAVEIS 

In [16]:
base_fraude_raw3 = base_fraude_raw2.copy()
base_fraude_raw3.to_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw3.csv')
base_fraude_raw3 = pd.read_csv(HOME_PATH+INTERIM_DATA_PATH+'base_fraude_raw3.csv')

## 3.1 - FILTRAGEM DE LINHAS 
Devido termos poucas colunas a quantidade de linhas farão toda a diferença, visto que não possuimos valores nulos e o valores `0` em algumas 

linhas são afirmações de `NÃO TRANSAÇÃO`, impactando assim, diretamente nos resultados finais.

## 3.2 - FILTRAGEM DE COLUNAS - SELEÇÃO DAS COLUNAS

Devido termos poucas colunas a quantidade de linhas farão toda a diferença, visto que não possuimos valores nulos e o valores `0` em algumas 

linhas são afirmações de `NÃO TRANSAÇÃO`, impactando assim, diretamente nos resultados finais.